In [1]:
import torchvision.transforms as T
from torch.utils.data import DataLoader

from torchvision.datasets.coco import CocoDetection as CocoDataset
from resnet import ResNet
from super_pycocotools.coco import SuperCOCO

from detectron2.config import get_cfg
from detectron2.data import build_detection_train_loader

In [2]:
super_ann_file = './datasets.json'
coco = SuperCOCO(super_ann_file)

processing DeepFruits
train data
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
processing SIXray
train data
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
test data
loading annotations into memory...
Done (t=0.08s)
creating index...
index created!


In [3]:
data_dir = coco.DeepFruits.infos['root']
data_type = 'train2017'
img_dir = '{}/{}/'.format(data_dir, data_type)

In [4]:
cfg = get_cfg()
cfg.DATASETS.TRAIN = coco.DeepFruits.register()[0]

In [49]:
cfg.SOLVER.IMS_PER_BATCH = 1

In [50]:
dataloader = build_detection_train_loader(cfg)

In [7]:
resnet = ResNet(cfg).cuda()

In [51]:
data = next(iter(dataloader))

In [9]:
for data in dataloader:
    outputs = resnet(data)
    break

{}
0


In [10]:
outputs.shape

torch.Size([2, 1024])

In [11]:
from detectron2.modeling import build_backbone

In [12]:
backbone = build_backbone(cfg)

In [13]:
backbone.to('cuda')

ResNet(
  (stem): BasicStem(
    (conv1): Conv2d(
      3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
      (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
    )
  )
  (res2): Sequential(
    (0): BottleneckBlock(
      (shortcut): Conv2d(
        64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
      )
      (conv1): Conv2d(
        64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
      (conv2): Conv2d(
        64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
      )
      (conv3): Conv2d(
        64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False
        (norm): FrozenBatchNorm2d(num_features=256, eps=1e-05)
      )
    )
    (1): BottleneckBlock(
      (conv1): Conv2d(
        256, 64, kernel_size=(1, 1), stride=(1, 1), bias=Fa

In [14]:
images = resnet.preprocess_image(data)
features = backbone(images.tensor)

{}
0


In [15]:
list(features)

['res4']

In [16]:
import torch.nn as nn

In [17]:
avgpool = nn.AdaptiveAvgPool2d(output_size=(1, 1))

In [18]:
cfg.MODEL.PROPOSAL_GENERATOR

CfgNode({'NAME': 'RPN', 'MIN_SIZE': 0})

In [19]:
features = [features[f] for f in features.keys()]

In [20]:
features[0].shape

torch.Size([2, 1024, 50, 68])

In [21]:
import torch
torch.cat(features).shape

torch.Size([2, 1024, 50, 68])

In [22]:
cfg.MODEL.RESNETS

CfgNode({'DEPTH': 50, 'OUT_FEATURES': ['res4'], 'NUM_GROUPS': 1, 'NORM': 'FrozenBN', 'WIDTH_PER_GROUP': 64, 'STRIDE_IN_1X1': True, 'RES5_DILATION': 1, 'RES2_OUT_CHANNELS': 256, 'STEM_OUT_CHANNELS': 64, 'DEFORM_ON_PER_STAGE': [False, False, False, False], 'DEFORM_MODULATED': False, 'DEFORM_NUM_GROUPS': 1})

In [23]:
output1 = resnet(data)
output2 = resnet(data)

{}
0
{}
0


In [24]:
from detectron2.checkpoint import DetectionCheckpointer

In [25]:
cfg.MODEL.WEIGHTS = "https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_50_C4_1x/137257644/model_final_721ade.pkl"

In [26]:
checkpointer = DetectionCheckpointer(resnet)
checkpointer.load(cfg.MODEL.WEIGHTS)

The checkpoint state_dict contains keys that are not used by the model:
  proposal_generator.rpn_head.conv.{bias, weight}
  proposal_generator.rpn_head.objectness_logits.{bias, weight}
  proposal_generator.rpn_head.anchor_deltas.{bias, weight}
  roi_heads.res5.0.shortcut.weight
  roi_heads.res5.0.shortcut.norm.{bias, running_mean, running_var, weight}
  roi_heads.res5.0.conv1.weight
  roi_heads.res5.0.conv1.norm.{bias, running_mean, running_var, weight}
  roi_heads.res5.0.conv2.weight
  roi_heads.res5.0.conv2.norm.{bias, running_mean, running_var, weight}
  roi_heads.res5.0.conv3.weight
  roi_heads.res5.0.conv3.norm.{bias, running_mean, running_var, weight}
  roi_heads.res5.1.conv1.weight
  roi_heads.res5.1.conv1.norm.{bias, running_mean, running_var, weight}
  roi_heads.res5.1.conv2.weight
  roi_heads.res5.1.conv2.norm.{bias, running_mean, running_var, weight}
  roi_heads.res5.1.conv3.weight
  roi_heads.res5.1.conv3.norm.{bias, running_mean, running_var, weight}
  roi_heads.res5.2.con

{'__author__': 'Detectron2 Model Zoo'}

In [52]:
output3 = resnet(data)

{}
0


In [29]:
(output2 == output1).all()

tensor(True, device='cuda:0')

In [28]:
(output2 == output3).all()

tensor(False, device='cuda:0')

In [54]:
torch.cat([output3.mean(0).unsqueeze(0)]).mean(0).shape

torch.Size([1024])